# Vergleich von dynamischem Stromtarif mit Fixpreistarif für einen Standardhaushalt

Solar und Windenergie -- bei allen Vorteilen -- schwanken stark in der Prodkution, was die Netze belastet. 
Dynamische Stromtarife wie z.B. [Tibber][] oder [Tado Hourly][] (ehemals aWATTar) gelten als Teil der Lösung, 
indem sie Anreize schaffen, Strom vorzugsweise dann zu verbrauchen, wenn er billig ist. Das ist in der Regel dann,
wenn die Sonne scheint und/oder der Wind weht. Bei dynamischen Stromtarifen bezahlt man keinen festen Preis pro 
Kilowattstunde. Stattdessen wird der stündliche Preis an der Strombörse festgelegt und immer um 14 Uhr für den Folgetag
bekannt gegeben. Der stündliche Preis kann nahezu null oder sogar negativ sein, bei Strommangel auch deutlich über
einem durchschnittlichen Fixpreis-Tarif liegen (theoretisch bis zu 4 EUR/kWh). Das ist natürlich besonders dann 
attraktiv, wenn man Großverbraucher hat, die sich gut automatisch steuern lassen (z.B. eine Wärmepumpe oder ein E-Auto). 

Nun haben wir (noch) keine solchen Großverbraucher, trotzdem hat mich interessiert wie teuer ein dynamischer 
Stromtarif im Vergleich zu einem Standardtarif wäre. Wir wohnen in einem Mehrfamilienhaus mit 4 Wohnungen, 
die sich seit 2024 einen Stromanschluss teilen. Der Verbrauch wid hausintern über Subzähler abgerechnet, die ich zeitgenau
[digital auslesen](https://github.com/grst/energymeter) kann. Dadurch habe ich für 2024 reale Verbrauchsdaten für 4 Wohnungen
mit unterschiedlichen Nutzungsprofilen:


| Wohnung | Beschreibung |
| -- | -- |
|Wohnung 1 | Ganzjährig bewohnt von einer alleinstehenden Person. Ca. 25 Jahre alte Haushaltsgeräte. Aus historischen Gründen hängen diverse gemeinschaftliche Verbraucher (Treppenhaus, Heizung, ...) zusätzlich auf diesen Zähler |
|Wohnung 2 | Ganzjährig bewohnt von zwei Personen. Teilweise erneuerte Haushaltsgeräte. Eigene 3kWp Photovolatikanlage mit 6kWh Speicher hinter dem Zähler. |
|Wohnung 3 | Ganzjährig bewohnt von einer Familie mit zwei kleinen Kindern. Neubauwohnung mit modernen Haushaltsgeräten. |
|Wohnung 4 | Sporadisch genutzte Ferienwohnung |

: Verbrauchsprofile vier Wohnungen {.striped .hover tbl-colwidths="[25,75]"}

Die stündlichen Verbrauchsdaten stelle ich auf GitHub öffentlich [zur Verfügung](https://github.com/grst/dynamischer-stromtarif/tree/b9dfb4b44b19819cc0d4748d48bc5cb7aa14022f/data) 
und werte sie im Folgenden aus.


## Verglichene Stromtarife

Es gibt eine Unzahl an unterschiedlichen Stromtarifen in Deutschland mit teilweise sehr unterschiedlichen
Preisstrukturen. Hier betrachte ich nur zwei Tarife: 

 * [AÜW Allgäustrom Basis](https://auew.de/privatkunden/strom/allgaeustrom-basis/), unser bisheriger Anbieter, Preisniveau für 2024
 * [Tado Hourly][]

Die Tarife gestalten sich wie folgt (alle Preise inkl. MwSt):

| Tarif | Arbeitspreis pro kWh | Grundpreis pro Monat |
| -- | -- | -- |
| AÜW | 32,19 ct (Strompreis + Steuern + Netzgebühr) | 15,68 EUR |
| tado | Epex Spot Day Ahead <br> + 19% MwSt <br> + 1,785ct (Aufschlag) <br> + 17,33116 ct (Netzgebühr + Steuern) | 15,77 EUR |

: Verglichene Stromtarife {.striped .hover tbl-colwidths="[25,50,25]"}


Die Grundpreise sind sehr ähnlich, daher betrachte ich nur die Arbeitspreise. 

[Tado Hourly]: https://energy.tado.com/
[Tibber]: https://tibber.com/de

## Vergleich für 2024

Es ergeben sich die folgenden jährlichen Gesamtkosten pro Wohneinheit und Tarif: 


In [1]:
#| results: hide
import pandas as pd
from datetime import datetime
import requests
import holoviews as hv
import locale

locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')
hv.extension("bokeh", logo=False)

In [2]:
VAT = 1.19  # VAT for dynamic prices
FIXED_FEE_PER_KWH = 0.19116  # EUR / kWh (brutto) (on top of dynamic price)
FEE_PER_KWH = 0.3322  # EUR / kWh (brutto), incl. network fee, for fixed price Tarif
YEARS = [2020, 2021, 2022, 2023, 2024]  # consider market prices from these years

hourly_cost = {}
monthly_cost = {}
yearly_cost = {}

In [3]:
# Get hourly prices from awattar API
begin = datetime.fromisoformat("2020-01-01").timestamp()
end = datetime.now().timestamp()
# API endpoint URL
url = f"https://api.awattar.de/v1/marketdata?start={int(begin)*1000}&end={int(end)*1000}"

# Fetch JSON data from the API
response = requests.get(url)
data = response.json()

# Convert JSON data to Pandas DataFrame
hourly_price = pd.DataFrame(data["data"])

hourly_price["start"] = pd.to_datetime(hourly_price["start_timestamp"], unit="ms")
hourly_price["end"] = pd.to_datetime(hourly_price["end_timestamp"], unit="ms")
hourly_price["marketprice"], hourly_price["unit"] = hourly_price["marketprice"] / 1000, "EUR/kWh"
hourly_price["start_day"] = hourly_price["start"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)
hourly_price["real_price"] = hourly_price["marketprice"] * VAT + FIXED_FEE_PER_KWH

# Load consumption data from table
pulses_interval = pd.read_csv("./data/verbrauch_2024.csv").rename(columns={"meter_name": "Zähler"})
pulses_interval["time"] = pd.to_datetime(pulses_interval["time"])
pulses_interval["start_day"] = pulses_interval["time"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)


In [4]:
#| include: false
# dynamic cost over the years

for year in YEARS:
    tmp_hourly_price = hourly_price.loc[
        lambda x: (x["start"] >= datetime.fromisoformat(f"{year}-01-01 00:00"))
        & (x["end"] < datetime.fromisoformat(f"{year+1}-01-01 00:00"))
    ]
    tmp_hourly_cost = pulses_interval.merge(
        tmp_hourly_price,
        how="inner",
        validate="many_to_one",
        left_on="start_day",
        right_on="start_day",
    ).assign(Kosten=lambda x: (x["real_price"] / 1000) * x["Wh"])

    tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
        lambda x: x.set_index("time")["Kosten"].resample("MS").sum()
    )
    tmp_yearly_cost = tmp_hourly_cost.loc[:, ["Zähler", "Kosten"]].groupby(["Zähler"]).sum()

    hourly_cost[f"tado_{year}"] = tmp_hourly_cost
    monthly_cost[f"tado_{year}"] = tmp_monthly_cost
    yearly_cost[f"tado_{year}"] = tmp_yearly_cost

/tmp/ipykernel_337393/2487920618.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
/tmp/ipykernel_337393/2487920618.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
/tmp/ipykernel_337393/2487920618.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping column

In [5]:
#| include: false
# fixed cost
tmp_hourly_cost = pulses_interval.merge(
    hourly_price, how="inner", validate="many_to_one", left_on="time", right_on="start"
).assign(Kosten=lambda x: (x["Wh"] / 1000) * FEE_PER_KWH)
tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(
    lambda x: x.set_index("time")["Kosten"].resample("MS").sum()
)
tmp_yearly_cost = tmp_hourly_cost.loc[:, ["Zähler", "Kosten"]].groupby(["Zähler"]).sum()

hourly_cost["AÜW_2024"] = tmp_hourly_cost
monthly_cost["AÜW_2024"] = tmp_monthly_cost
yearly_cost["AÜW_2024"] = tmp_yearly_cost

/tmp/ipykernel_337393/2392160184.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp_monthly_cost = tmp_hourly_cost.groupby(["Zähler"]).apply(


In [6]:
yearly_cost_df = pd.concat(
    [df.assign(Tarif=k.split("_")[0]) for k, df in yearly_cost.items() if "2024" in k]
).sort_values(["Zähler", "Tarif"])

In [7]:
#| tbl-cap: Jährliche Stromkosten in EUR für AÜR und tado pro Wohneinheit.

yearly_cost_df.pivot_table(values="Kosten", columns="Tarif", index="Zähler").assign(
    **{
        "Ersparnis (EUR)": lambda x: -(x["tado"] - x["AÜW"]),
        "Ersparnis (%)": lambda x: (1 - x["tado"] / x["AÜW"]) * 100,
    }
).style.format('{:.2f}')

Tarif,AÜW,tado,Ersparnis (EUR),Ersparnis (%)
Zähler,,,,
Wohnung 1,1155.47,1003.21,152.26,13.18
Wohnung 2,704.56,626.17,78.39,11.13
Wohnung 3,574.34,508.87,65.47,11.40
Wohnung 4,172.87,155.06,17.81,10.30


Für 2024 wäre *tado hourly* also
je nach Wohneinheit zwischen 10% und 13% günstiger gewesen als der Fixpreistarif von AÜW. Möglicherweise gäbe es aber
auch andere, günstigere Fixpreistarife, die eine vergleichbare Einsparung bieten. Die Einsparung wurde aber bereits ohne
intelligente Steuerung von Verbrauchern bzw. enstprechenden Verhaltensänderungen erzielt. Würde man Strom bevorzugt
zu günstigen Tageszeiten nutzen, ließe sich ggf. deutlich mehr einsparen. 

In [8]:
monthly_cost_df = (
    pd.concat([df.assign(Tarif=k.split("_")[0]) for k, df in monthly_cost.items() if "2024" in k])
    .sort_values(["Zähler", "Tarif"])
    .reset_index()
    .melt(["Zähler", "Tarif"], value_name="Kosten")
    .assign(month=lambda x: pd.to_datetime(x["time"]).dt.strftime("%b"))
    .assign(month=lambda x: pd.Categorical(x["month"], categories=x["month"].unique()))
)

Die folgende Grafik zeigt den Vergleich monatsweise: 

In [9]:
hv.Table(monthly_cost_df, ["Tarif", "Zähler", "month"], ["Kosten"]).to.bars(["month", "Tarif"], "Kosten").opts(
    width=450, xrotation=90, legend_position="right"
)

:HoloMap   [Zähler]
   :Bars   [month,Tarif]   (Kosten)

Das Einsparpotenzial scheint unabhängig vom Monat, lediglich im November und Dezember 2024 sind die Kosten von AÜR und tado vergleichbar.

## Vergleich mit den Vorjahren

Aufgrund [der Energiekrise 2021-2023](https://de.wikipedia.org/wiki/Globale_Energiekrise_2021%E2%80%932023) sind die Strompreise
im Jahr 2022 stark gestiegen. Daher hat mich interessiert, wie der dynamische Stromtarif in diesen Jahren abgeschnitten hätte. 

Bei der Interpretation der Ergebnisse sind folgende Limitierungen zu berücksichtigen:

 * Mir liegen nur die Verbrauchsdaten für 2024 vor. Ich wende also die Preise aus den Vorjahren auf die Verbrauchsdaten von 2024 an.
 * Auch die Fixpreistarife wurden je nach Anbieter und Vertrag in den Krisenjahren teilweise stark erhöht.
 * Die Netznutzungsgebühren waren in den Vorjahren geringer als in 2024. Das ist hier nicht berücksichtigt.
 * 2023 wurde in Deutschland [die Strompreisbremse](https://de.wikipedia.org/wiki/Strompreisbremse) eingeführt. Diese [war auch auf dynamische Stromtarife anwendbar](https://web.archive.org/web/20231107120938/https://tibber.com/de/strompreisbremse). 

Die folgende Grafik zeigt die hypothetischen jährliche Gesamtkosten für tado hourly pro Wohneinheit für die Jahre 2020 bis 2024 (Verbrauchsdaten und Netzentgelt von 2024). 

In [10]:
yearly_cost_df = pd.concat(
    [df.assign(year=k.split("_")[1], Tarif=k.split("_")[0]) for k, df in yearly_cost.items()]
).sort_values(["Zähler", "year", "Tarif"])

In [11]:
hv.Table(yearly_cost_df, ["Tarif", "Zähler", "year"], "Kosten").to.bars(["year", "Tarif"], "Kosten").opts(
    width=400, xrotation=90, legend_position="right"
)

:HoloMap   [Zähler]
   :Bars   [year,Tarif]   (Kosten)

Im Krisenjahr 2022 wäre es also zu einem deutlichen Preisanstieg gekommen. Allerdings sind die Marktpreise in 2023 bereits wieder gefallen, während [viele Fixpreisanbieter ihre Preise teils deutlich erhöht haben](https://www.businessinsider.de/wirtschaft/verbraucher/diese-680-stromanbieter-haben-2023-die-preise-erhoeht-die-tabelle-zeigt-euch-wer-am-meisten-draufgeschlagen-hat-i/).

In [12]:
#| include: false
monthly_cost_df = (
    pd.concat([df.assign(Tarif=k) for k, df in monthly_cost.items()])
    .sort_values(["Zähler", "Tarif"])
    .reset_index()
    .melt(["Zähler", "Tarif"], value_name="Kosten")
    .assign(month=lambda x: pd.to_datetime(x["time"]).dt.month_name(locale="de_DE.utf8"))
    .assign(month=lambda x: pd.Categorical(x["month"], categories=x["month"].unique()))
)


# Ratio to fixed Tarif over the years
def get_ratios(df):
    ratios = []

    for year in YEARS:
        ratio = pd.Series(
            df.loc[lambda x: x["Tarif"] == f"tado_{year}", "Kosten"].values
            / df.loc[lambda x: x["Tarif"] == "AÜW_2024", "Kosten"].values,
            name=str(year),
        )
        ratios.append(ratio)
    return pd.DataFrame(ratios)


monthly_cost_ratio = (
    monthly_cost_df.groupby(["Zähler", "month"])
    .apply(get_ratios)
    .reset_index()
    .rename(columns={"level_2": "year", 0: "ratio"})
)


/tmp/ipykernel_337393/3351810925.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_cost_df.groupby(["Zähler", "month"])
/tmp/ipykernel_337393/3351810925.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_ratios)


Die folgende Grafik zeigt das Preisverhältnis zum AÜW Tarif von 2024, nach Monaten. Werte >1 bedeuten dass der dynamische
Stromtarif im entsprechenden Monat teurer gewesen wäre als der Fixpreistarif.  

In [13]:
hv.HLine(1).opts(color="black", line_width=2, line_dash="dashed") * hv.Table(
    monthly_cost_ratio, ["year", "Zähler", "month"], ["ratio"]
).to.curve("month", "ratio").overlay("year").opts(width=450, height=400, xrotation=45, legend_position="top")

:HoloMap   [Zähler]
   :Overlay
      .HLine.I     :HLine   [x,y]
      .NdOverlay.I :NdOverlay   [year]
         :Curve   [month]   (ratio)

## Abschließende Betrachtungen

 * Der dynamische Stromtarif birgt ein gewisses Preisrisiko. Im Fall von Krisen kann der Stompreis schnell steigen. Allerdings sinken die Preise auch schneller wieder. Unterm Strich macht es über die Jahre wohl wenig Unterschied. 
 * Ohne steuerbare Großverbraucher ist das Einsparpotential im Bereich von wenigen Euro pro Monat und Wohneinheit. Wohl kaum genug um den durchschnittlichen Verbraucher zu einer Änderung zu bewegen.
 * Die Netznutzungsgebühren werden als Festpreis pro kWh berechnet -- das verwässert das Preissignal. Um den Anreiz zu erhöhen, den Stromverbrauch "smart" zu steuern wäre es sinnvoller die Gebühren abhängig vom Strompreis zu machen.
 * Strompreise sind ein komplexes Thema und ein korrekter Vergleich ist schwierig. Strompreise und 
   Netzentgelte ändern sich laufend, auch die von Fixpreistarifen. Dazu gibt es noch [Tarife mit reduzierten Netzentgelten](https://www.allgaeunetz.com/netznutzungsentgelte.html), wenn der Netzbetreiber Großverbraucher bei Bedarf abschalten/drosseln darf.
 * Wir haben 2024 eine 24kWp Photovoltaikanlage mit 22kWh Stromspeicher installiert. Es bedarf eine separaten Analyse ob ein dynamischer Stromtarif in Kombination mit einer PV Analge noch Vorteile bietet. Wenn man gerade selber keinen Strom produziert, ist er vermutlich auch gerade teuer. Andererseits sind die Ergebnisse von Wohnung 2 (mit eigener kleiner PV Anlage) vergleichbar mit den Wohnungen ohne. Und evtl. kann man im Winter die [Hausbatterie mit günstigem Nachtstrom laden](https://docs.evcc.io/docs/features/battery#netzladen).  